# 사이킷런으로 시작하는 머신러닝 과제
(2조 πsun)

## 1. 개념

### 1-1. 각 문장이 맞는지 틀린지를 T/F로 표시하시오.
1. 사이킷런은 파이썬 머신러닝 라이브러리 중 가장 많이 사용되는 라이브러리이며 다른 머신러닝 패키지도 사이킷런 스타일 API를 지향한다. (  T  )
2. 분류는 대표적인 비지도 학습의 방법 중 하나이다. (  F  )
3. 지도학습은 결과가 있는 데이터로 학습한 뒤 새로운 데이터를 가지고 결과를 예측하는 방법이다. (  T  )
4. K-Fold 교차 검증은 K개의 데이터 폴드 세트를 만들어 K번만큼 각 fold 세트데이터 학습과 검증 평가를 반복적으로 수행하는 방법이다. (  T  )
5. Stratified K 폴드는 균형 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식이다. (  F  )

### 1-2. 아래 빈칸을 채우시오.
1. ( 하이퍼 파라미터 )란 머신러닝 알고리즘별로 최적의 학습을 위해 직접 입력하는 파라미터를 말한다.
2. 학습 데이터와 테스트 데이터로 분리하는 데 ( train_test_split )함수를 사용한다.
3. 학습 데이터로 학습된 모델의 성능을 평가하려면 ( 테스트 데이터 )가 필요하다.
4. ( Estimator )란 분류 알고리즘을 구현한 클래스 Classifier와 회귀 알고리즘을 구현한 클래스를 Regressor을 합친 클래스를 말한다.
5. ( 오버피팅 )란 학습 데이터에만 과도하게 최적화된 것을 말하며 ( 교차검증 )을 하여 이를 예방한다.

### 1-3. 붓꽃 데이터 세트를 분류하는 예측 프로세스 과정을 순서대로 나열하시오.
1. 모델 학습
2. 평가
3. 데이터 세트 분리
4. 예측 수행    

정답 : (  3  -  1  -  4  -  2  )

## 2. 실습
다음은 위스콘신 유방함 피처들과 악성/음성 레이블 데이터 세트를 분류하는 실습 과제이다. 
아래는 데이터 이해를 위한 변수에 대한 간략한 설명을 담고 있다.     
![위스콘신_데이터_설명](./wisc_cancer.png)

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,cross_val_score,GridSearchCV,train_test_split
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd

In [3]:
breast_cancer=load_breast_cancer()    # 데이터 세트 불러옴
features=breast_cancer.data           # feature 데이터 값을 features에 저장
label=breast_cancer.target            # label 데이터 값을 label에 저장
dt_clf=DecisionTreeClassifier(random_state=100)    # 의사 결정 트리 객체 생성(seed는 100으로 고정)

### 2-1. 다음 데이터를 이용하여 Stratified K폴드와 cross_val_score() 값비교하기

2-1-1. Stratified K폴드이용 (폴드수는 5개로 설정)

In [4]:
skfold=StratifiedKFold(n_splits=5)    # fold수를 5개로 하는 StratifiedKFold 객체 생성
cv_accuracy=[]    # 교차 검증 정확도를 저장하는 리스트

for train_index,test_index in skfold.split(features, label):
    x_train, x_test=features[train_index], features[test_index]
    y_train, y_test=label[train_index], label[test_index]
    dt_clf.fit(x_train,y_train)    # 학습 
    pred=dt_clf.predict(x_test)    # 현재 폴드 데이터 세트에 대한 평가

    accuracy=np.round(accuracy_score(y_test, pred),4)    # 예측값과 결과값을 비교하여 정확도를 계산(소수점 넷째자리까지)
    cv_accuracy.append(accuracy)                         # cv_accuracy에 현재 폴드 데이터 세트의 정확도 추가
    
print('\n 교차 검증별 정확도:',np.round(cv_accuracy,4))
print(' 평균 검증 정확도:',np.round(np.mean(cv_accuracy),4))    # 모든 폴드 데이터 세트의 정확도의 평균


 교차 검증별 정확도: [0.913  0.913  0.9027 0.9292 0.9115]
 평균 검증 정확도: 0.9139


답:교차 검증별 정확도 : [0.913  0.913  0.9027 0.9292 0.9115]    
평균 검증 정확도 : 0.9139

2-1-2. cross_val_score()이용
(폴드수는 5개로 설정)

In [5]:
# 위에서 교차 검증을 구하는 과정을 cross_val_score() 함수로 대체할 수 있다.
scores =cross_val_score(dt_clf,features,label,scoring='accuracy',cv=5)
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도:' ,np.round(np.mean(scores),4))

교차 검증별 정확도: [0.913  0.913  0.9027 0.9292 0.9115]
평균 검증 정확도: 0.9139


답:교차 검증별 정확도 : [0.913  0.913  0.9027 0.9292 0.9115]    
평균 검증 정확도 : 0.9139

### 2-2. GridSearchCV를 이용하여 정확도 구하기
(max_depth는 1,2,3,4,5/min_samples_split은 2,3,4,5/ 폴드수는 6개/TEST_SIZE는 0.1로 각각 설정)

In [6]:
# 테스트 데이터의 비율이 10%로 하여 데이터 세트를 분리
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=100)
parameters={'max_depth':[1,2,3,4,5], 'min_samples_split':[2,3,4,5]}     # 테스트할 하이퍼 파라미터 세트

# 하이퍼 파라미터를 6개의 train, test set fold로 나누어 테스트 수행 설정
grid_dt_clf=GridSearchCV(dt_clf,param_grid=parameters,cv=6,refit=True)

# param_grid의 하이퍼 파라미터를 순차적으로 학습 및 평가
grid_dt_clf.fit(x_train,y_train)

print('GridSearchCV 최적 파라미터:' ,grid_dt_clf.best_params_)
print('GridSearchV 최고정확도:{0:.4f}'.format(grid_dt_clf.best_score_))

# GridSearchCV의 refit으로 최적 성능을 나타내는 하이퍼 파라미터로 학습된 best_estimator_를 estimator에 저장
estimator = grid_dt_clf.best_estimator_

# 예측값과 실제값 비교
pred = estimator.predict(x_test)
print('테스트 데이터 세트 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))

GridSearchCV 최적 파라미터: {'max_depth': 4, 'min_samples_split': 4}
GridSearchV 최고정확도:0.9375
테스트 데이터 세트 정확도:0.9298


답 : GridSearchCV 최적 파라미터: {'max_depth': 4, 'min_samples_split': 4}    
GridSearchV 최고정확도 : 0.9375    
테스트 데이터 세트 정확도 : 0.9298